# 텍스트 전처리 (Text Preprocessing)

credit @ 김채형

1. 문장 토큰화
2. 단어 토큰화
3. 형태소 분석
4. 품사 태깅
5. 어간 추출 & 원형 복원
6. 불용어 제거

## 1. 문장 토큰화 (Sentence Tokenization) ; 문장 분류 (Sentence Segmentation)

문장 토큰화는 토큰의 단위를 문장으로 하여, 코퍼스 내 텍스트를 문장 단위로 구분하는 작업을 의미합니다. 텍스트를 문장 단위로 구분할 때에는 기본적으로 온점(.), 물음표(?), 느낌표(!) 등을 기준으로 합니다. 하지만, 꽤 정확하게 문장 구분자(boundary)로서의 역할을 수행하는 물음표 및 느낌표와는 달리, 온점(.)은 꼭 문장의 끝이 아니더라도 등장할 수 있기 때문에 어려움이 존재합니다. 예를 들어 다음과 같은 코퍼스가 있다고 가정해봅시다.

ex1) IP 192.168.56.31 서버에 들어가서 로그 파일 저장해서 ukairia777@gmail.com로 결과 좀 보내줘. 그러고나서 점심 먹으러 가자.  
ex2) Since I'm actively looking for Ph.D. students, I get the same question a dozen times every year.

ex1의 경우, 문장 토큰화 결과 "IP 192.168.56.31 서버에 들어가서 로그 파일 저장해서 ukairia777@gmail.com로 결과 좀 보내줘.", "그러고나서 점심 먹으러 가자." 두개의 문장으로 구분되는 것이 가장 적절합니다. ex2의 경우, 문장 토큰화 결과 "Since I'm actively looking for Ph.D. students, I get the same question a dozen times every year." 한개의 문장으로 구분되는 것이 가장 적절합니다. 하지만 단순히 온점만을 기준으로 문장을 구분한다면, 이와 같이 적절하게 문장 토큰화가 진행되지 않을 것입니다. 그렇기 때문에 코퍼스가 어떤 국적의 언어인지, 코퍼스 내에서 특수문자들이 어떻게 사용되고 있는지 등에 따라 문장 토큰화 규칙들을 정의할 필요가 있습니다.

영어의 경우 NLTK의 `sent_tokenize`를 사용하여 영어 문장 토큰화를 수행할 수 있습니다.

In [1]:
from nltk.tokenize import sent_tokenize
text = 'His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to mae sure no one was near.'
print(sent_tokenize(text))

['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the edge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to mae sure no one was near.']


In [2]:
# 문장 중간에 .이 있는 경우
from nltk.tokenize import sent_tokenize
text = 'I am actively looking for Ph.D. students. and you are a Ph.D student.'
print(sent_tokenize(text))

['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']


한국어의 경우 박상길님이 개발한 KSS(Korean Sentence Splitter)를 통해 한국어 문장 토큰화를 수행할 수 있습니다.

In [3]:
import kss
text = "딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어려워요. 농담아니에요. 이제 해보면 알걸요?"
print(kss.split_sentences(text))

## 2. 단어 토큰화 (Word Tokenization)

### 2.1 개념

단어 토큰화는 토큰의 단위를 단어로 하여, 코퍼스 내 텍스트를 단어 단위로 구분하는 작업을 의미합니다. 이때 단어는 single word 이외에도 단어구 등 의미를 갖는 단위를 포괄합니다. 영어의 경우 텍스트를 단어 단위로 구분할 때 보통 띄어쓰기 즉 공백(whitespace)을 기준으로 합니다. 예를 들어 다음과 같은 코퍼스가 있다고 가정해봅시다.

코퍼스 : Time is an illusion. Lunchtime double so!

이에 대하여 구두점을 제거하고 띄어쓰기를 기준으로 단어를 구분하면 다음과 같습니다.

"Time", "is", "an", "illustion", "Lunchtime", "double", "so"

이를 가장 기초적인 단어 토큰화라고 할 수 있습니다. 하지만 현실에서 단어 토큰화를 수행할 때에는 훨씬 더 많은 고려해야 할 사항들이 존재합니다.

한국어의 경우에는 텍스트를 단어 단위로 구분할 때 띄어쓰기만을 기준으로 하면 단어 토큰화가 제대로 수행되지 않습니다. 왜냐하면 근본적으로 한국어는 교착어이고, 영어에 비해 띄어쓰기가 잘 지켜지지 않는 경향이 있기 때문입니다.

### 2.2 단어 토큰화에서 고려해야 할 사항

굉장히 다양한 상황들이 존재하기 때문에, 단어 토큰화를 단지 구두점을 제거하고 공백을 기준으로 단어를 구분하는 작업이라고 여길 수는 없습니다. 단어 토큰화를 수행할 때에는 여러 고려사항이 존재합니다.

**구두점 및 특수 문자**

코퍼스에 대해서 단어 토큰화를 수행하고자 할 때, 구두점 혹은 특수문자를 무조건 제거하는 것은 바람직하지 않습니다. 이때 구두점(punctuation)은 온점(.), 쉼표(,), 물음표(?), 느낌표(!) 세미콜론(;) 등과 같은 기호를 의미합니다. 온점(.)의 경우, 기본적으로 문장의 경계를 알 수 있는 문장 구분자(boundary)로서의 역할을 합니다. 또 "Ph.D"와 같이 단어 자체에 구두점이 포함된 경우도 존재합니다. 특수문자 중 \\$의 경우 "$45.55"와 같이 가격을 의미하기도 하고, /의 경우 "01/02/06"와 같이 날짜를 의미하기도 합니다. 이렇게 다양한 상황들이 존재하기 때문에 구두점과 특수문자를 무조건적으로 제거하는 것은 바람직하지 않습니다.

**줄임말**

영어의 경우 아포스트로피(')는 단어를 줄임말로 쓸 때 사용됩니다. 예를 들어, "we're"은 "we are", "I'm"은 "I am"의 줄임말인데, 이때 "re"와 "m"을 접어(clitic)라고 합니다. 예를 들어 다음과 같은 코퍼스에 대해서 단어 토큰화를 수행한다고 가정해봅시다.

코퍼스 : Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop.

이때 아포스트로피를(')가 존재하는 단어 "Don't"와 "Jone's"에 대하여 어떻게 토큰으로 구분할 것인지의 문제가 존재합니다.

In [4]:
# word_tokenize
from nltk.tokenize import word_tokenize
text = "Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."
print(word_tokenize(text))

['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


Don't => Do 와 n't 로 구분  
Jone's => Jone 와 's로 구분 

In [5]:
# WordPunctTokenizer : 구두점을 별도의 토큰으로 구분
from nltk.tokenize import WordPunctTokenizer
text = "Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."
print(WordPunctTokenizer().tokenize(text))

['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


Don't => Don 와 ' 와 t 로 구분  
Jone's => Jone 와 ' 와 s 로 구분

**단어 내에 공백이 존재하는 경우**

예를 들어, "New York"이나 "rock n roll"과 같은 단어의 경우에는 한 단어임에도 불구하고 단어 내에 공백이 존재합니다. 단어 토큰화는 이러한 단어들을 한 단어 즉 한 토큰으로 인식할 수 있어야 합니다.

### 2.3 Penn Treebank Tokenization

Penn Treebank Tokenization은 표준으로 쓰이고 있는 토큰화 방법 중 하나입니다. Penn Treebank Tokenization의 규칙은 다음과 같습니다.

규칙 1. 하이푼으로 구성된 단어는 하나로 유지한다.  
규칙 2. 아포스트로피로 접어가 함께 하는 단어는 분리한다.

아래와 같은 코퍼스에 대하여 Penn Treebank Tokenization을 수행해보고자 합니다.

코퍼스 : "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."

In [6]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
text = "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."
print(tokenizer.tokenize(text))

['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


규칙1에 따라 "home-based"의 경우 하나의 토큰으로 인식합니다.  
규칙2에 따라 "dosen't"의 경우 "does"와 "n't"로 분리합니다.

## 3. 형태소 분석

앞서 언급했듯이, 단어 토큰화는 토큰의 단위를 단어로 하여, 코퍼스 내 텍스트를 단어 단위로 구분하는 작업을 의미합니다. 영어의 경우 단어 토큰화를 수행할 때 띄어쓰기를 단어 구분 기준으로 하는데, 이를 어절 토큰화라고 합니다. 그런데 한국어의 경우 단어 토큰화를 수행할 때 어절 토큰화를 사용하는 것은 부적절합니다. 이는 한국어가 교착어라는 점에 기인합니다. 대신, 한국어의 경우 단어 토큰화를 수행할 때 토큰의 단위를 형태소로 하는 형태소 토큰화를 사용합니다.

**교착어**

한국어는 교착어입니다. 교착어란 조사, 어미 등을 붙여서 말을 만드는 언어를 의미합니다. 예를 들어, "그"라는 단어에 다양한 조사를 붙여서  "그가", "그에게", "그를", "그와" 등의 말을 만들 수 있습니다. 이때 조사는 "그"라는 단어 뒤에 띄어쓰기 없이 바로 붙게 됩니다. 즉 한국어는 어절이 독립적인 단위로 구성되지 않습니다. 조사 등의 무언가가 붙어있는 경우가 많기 때문입니다. 따라서 어절 토큰화를 사용할 수 없습니다. NLP에서는 단어의 형태가 달라 같은 단어임에도 다른 단어인 것처럼 취급되는 것이 큰 문제로 작용합니다. 따라서 한국어에서는 조사, 어미 등을 분리해야 할 필요가 있습니다. 이를 위해서는 토큰의 단위를 형태소로 하는 형태소 토큰화를 사용해야 합니다.

**형태소**

형태소(morpheme)란 뜻을 가진 가장 작은 말의 단위를 의미합니다. 형태소는 자립 형태소와 의존 형태소로 나뉩니다.

- 자립 형태소 : 접사, 어미, 조사와 상관없이 자립하여 사용할 수 있는 형태소 : 체언 (명사, 대명사, 수사), 수식언 (관형사, 부사), 감탄사 등 => 그 자체로 단어
- 의존 형태소 : 다른 형태소와 결합하여 사용되는 형태소 : 조사, 접사, 어간, 어미

예를 들어 아래와 같은 문장에 대하여 형태소 분석을 하면 다음과 같습니다.

문장 : 에디가 딥러닝책을 읽었다  
자립 형태소 : 에디, 딥러닝책  
의존 형태소 : -가, -을, 읽-, -었, -다

**형태소 분석기**

한국어의 경우 KoNLPy를 활용하여 단어 토큰화가 가능합니다. KoNLPy에서 제공하는 형태소 분석기는 다음과 같습니다.

- Okt (Open Korea Text)
- 매캡 (Mecab)
- 코모란 (Komoran)
- 꼬꼬마 (Kkma)
- 한나눔 (Hannanum)

형태소 분석기를 사용하여 단어 토큰화를 수행한다는 것은 형태소 토큰화를 수행한다는 것입니다.

## 4. 품사 태깅 (Part-Of-Speech Tagging ; POS Tagging)

때때로 단어는 표기는 같지만 품사에 따라 단어의 의미가 달라지는 경우가 발생합니다.

예를 들어, 영어의 경우, 단어 "fly"는 동사로 쓰일 때에는 '날다'라는 의미를 갖는 한편 명사로 쓰일 때에는 '파리'라는 의미를 갖습니다. 한국어의 경우, "못"이라는 단어는 명사일 때에는 망치를 사용해서 목재 따위를 고정하는 물건이라는 의미로 쓰이는 한편 부사일 때에는 동작 동사를 할 수 없다는 의미로 쓰입니다.

결과적으로, 이는 단어의 의미를 제대로 파악하기 위해서는 해당 단어의 품사 정보가 필요하다는 것을 시사합니다. 이에 따라 단어 토큰화 과정에서 각 단어가 어떤 품사로 쓰였는지 구분해놓기도 하는데, 이를 품사 태깅(Part-Of-Speech tagging ; POS Tagging)이라고 합니다.

**Penn Treebank**

|Number|Tag|Description|
|------|---|-----------|
|1.|CC |Coordinating conjunction|
|2.|CD |Cardinal number|
|3.|DT |Determiner|
|4.|EX |Existential there|
|5.|FW |Foreign word|
|6.|IN |Preposition or subordinating conjunction|
|7.|JJ |Adjective|
|8.|JJR|Adjective, comparative|
|9.|JJS|Adjective, superlative|
|10.|LS|List item marker|
|11.|MD|Modal|
|12.|NN|Noun, singular or mass|
|13.|NNS|Noun, plural|
|14.|NNP|Proper noun, singular|
|15.|NNPS|Proper noun, plural|
|16.|PDT|Predeterminer|
|17.|POS|Possessive ending|
|18.|PRP|Personal pronoun|
|19.|PRP\\$|Possessive pronoun|
|20.|RB |Adverb|
|21.|RBR|Adverb, comparative|
|22.|RBS|Adverb, superlative|
|23.|RP	|Particle|
|24.|SYM|Symbol|
|25.|TO	|to|
|26.|UH	|Interjection|
|27.|VB	|Verb, base form|
|28.|VBD|Verb, past tense|
|29.|VBG|Verb, gerund or present participle|
|30.|VBN|Verb, past participle|
|31.|VBP|Verb, non-3rd person singular present|
|32.|VBZ|Verb, 3rd person singular present|
|33.|WDT|Wh-determiner|
|34.|WP	|Wh-pronoun|
|35.|WP$|Possessive wh-pronoun|
|36.|WRB|Wh-adverb|

**세종 품사 태그**

|대분류|태그|설명|
|----|---|---|
|체언|NNG|일반 명사|
|체언|NNP|고유 명사|
|체언|NNB|의존 명사|
|체언|NR |수사|
|체언|NP |대명사|
|용언|VV |동사|
|용언|VA |형용사|
|용언|VX |보조 용언|
|용언|VCP|긍정 지정사|
|용언|VCN|부정 지정사|
|관형사|MM|관형사|
|부사|MAG|일반 부사|
|부사|MAJ|접속 부사|
|감탄사|IC|감탄사|
|조사|JKS|주격 조사|
|조사|JKC|보격 조사|
|조사|JKG|관형격 조사|
|조사|JKO|목적격 조사|
|조사|JKB|부사격 조사|
|조사|JKV|호격 조사|
|조사|JKQ|인용격 조사|
|조사|JX |보조사|
|조사|JC |접속 조사|
|선어말 어미|EP|선어말 어미|
|어말 어미|EF|종결 어미|
|어말 어미|EC|연결 어미|
|어말 어미|ETN|명사형 전성 어미|
|어말 어미|ETM|관형형 전성 어미|
|접두사|XPN|체언 접두사|
|접미사|XSN|명사 파생 접미사|
|접미사|XSV|동사 파생 접미사|
|접미사|XSA|형용사 파생 접미사|
|어근|XR|어근|
|부호|SF|마침표, 물음표, 느낌표|
|부호|SP|쉼표, 가운뎃점, 콜론, 빗금|
|부호|SS|따옴표, 괄호표, 줄표|
|부호|SE|줄임표|
|부호|SO|붙임표 (물결, 숨김, 빠짐)|
|부호|SW|기타기호 (논리수학기호, 화폐기호)|
|분석 불능|NF|명사추정범주|
|분석 불능|NV|용언추정범주|
|분석 불능|NA|분석불능범주|
|한글 이외|SL|외국어|
|한글 이외|SH|한자|
|한글 이외|SN|숫자|

## 5. 어간 추출 (Stemming) & 원형 복원 (Lemmatization)

단어의 형태 변화(lexical variations of term ; term variation) 에 따라 같은 단어라도 다른 단어인 것처럼 취급되는 문제를 해결하기 위해 사용되는 보편적인 방법으로 어간 추출(Stemming)과 원형 복원(Lemmatization)이 있습니다.

### 5.1 Stemming (어간 추출)

Stemming이란 어형이 변형된 단어로부터 접사 등을 제거하고 그 단어의 어간을 분리해내는 것을 의미합니다. 이때 어간이 반드시 어근과 같아야 하는 것은 아니며, Stemming의 목적은 어근과 차이가 있더라도 관련이 있는 단어들이 일정하게 동일한 어간으로 매핑되게 하는 것입니다. 이러한 역할을 하는 것을 Stemming Algorithm 또는 Stemmer라고 합니다.

stemming은 정보검색 분야에서 처음 등장했습니다. 정보검색시스템의 색인 과정에서, 단어의 prefix에 variation이 많은 경우 질의어와 색인어 간에 매칭이 잘 이루어지지 않아 질의어도 stemming을 하고 색인어도 stemming을 하여 검색 성능을 높이고자 하였습니다. 많은 웹 검색엔진들은 동일한 어간을 가진 단어들을 동의어로 취급하는 방식으로 질의어 확장을 하여 검색 결과의 품질을 높입니다.

예를 들어, 'automate', 'automatic', 'automation' 이렇게 세개의 단어가 있다고 가정해봅시다. 이들은 각각 'e', 'ic', 'ion'이라는 접사를 가지고 있어 어형이 변형되었지만 모두 'automat'라는 어간을 가지고 있습니다. 이러한 단어들에 대하여 접사를 제거하고 동일한 어간인 'automat'으로 매핑되도록 하는 작업이 stemming입니다.

대표적인 Stemming Algorithm으로 Martin Porter가 고안한 Porter Stemming Algorithm = Porter Stemmer 가 있습니다. rule base로, 많은 규칙들에 기반하여 어미를 분리 및 제거함으로써 어간을 추출해줍니다. Porter Stemming Algorithm은 aggressive하지 않다는 점에서 편리합니다. aggressive한 stemming algorithm의 경우에는 단어로부터 더 많은 부분을 제거하여 더 적은 부분만을 어간으로 남깁니다. 그 결과 term variation 때문에 다른 단어인 것처럼 취급된 것이 아니라 완전히 다른 단어임에도 불구하고 전혀 다른 단어들이 하나의 어간으로 처리되는 경우가 발생합니다. 이러한 점을 방지하는 데에 Porter Stemming Algorithm이 가장 효과적입니다.

stemming은 DTM의 dimension reduction에 효과적이라는 장점을 갖습니다. Zipf's law에 따르면, 단어들의 출현빈도는 long tail이기 때문에 문서집단 내에는 극저빈도의 단어들이 많습니다. 이러한 단어들의 term variation을 잡아줌으로써 하나의 단어로 통일하여 의미를 보존한 채 DTM의 차원을 축소할 수 있습니다. 힌편 앞서 언급한 것과 같이, stemming은 뜻을 구분할 수 없을 정도로 어간이 짧게 잡히는 경우가 발생할 수 있다는 단점을 갖습니다. 또한 stemming을 할 경우 품사 정보를 알 수 없어 문제가 되는 경우도 존재합니다.

In [7]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

text = "This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes."

# word tokenization
words = word_tokenize(text)

# stemming
s = PorterStemmer()
result = [s.stem(w) for w in words]

# 결과 출력
print(text)
print(words)
print(result)

This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes.
['This', 'was', 'not', 'the', 'map', 'we', 'found', 'in', 'Billy', 'Bones', "'s", 'chest', ',', 'but', 'an', 'accurate', 'copy', ',', 'complete', 'in', 'all', 'things', '--', 'names', 'and', 'heights', 'and', 'soundings', '--', 'with', 'the', 'single', 'exception', 'of', 'the', 'red', 'crosses', 'and', 'the', 'written', 'notes', '.']
['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', "'s", 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'thing', '--', 'name', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


In [8]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

# stemming
s = PorterStemmer() # 포터 스태머
l = LancasterStemmer() # 랭커스터 스태머
ss = [s.stem(w) for w in words]
ll = [l.stem(w) for w in words]

# 결과 출력
print(words)
print(ss)
print(ll)

['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
['polici', 'do', 'organ', 'have', 'go', 'love', 'live', 'fli', 'die', 'watch', 'ha', 'start']
['policy', 'doing', 'org', 'hav', 'going', 'lov', 'liv', 'fly', 'die', 'watch', 'has', 'start']


### 5.2 Lemmatization (원형 복원 ; 표제어 추출)

Lemmatization은 한 단어가 여러 형식으로 표현되어 있는 것을 단일 형식으로 묶어주는 기법입니다. 예를 들어, 'am', 'are', 'is' 세개의 단어에 대하여 lemmatization을 실시할 경우 그 결과는 'be'가 됩니다.

단어의 의미적 단위를 고려하지 않고 기계적으로 경험적 법칙에 의해 결정되는 stemming과 달리, Lemmatization은 단어의 의미적인 단위를 고려하고 자연어 처리 기법의 근간이 되는 형태소 분석을 통해 이루어지기 때문에 더 정확한 단어 수준 분석을 수행할 수 있습니다. 즉 Lemmatization을 수행할 경우, 품사 정보가 남아있기 때문에 의미론적 관점에서 더 효과적입니다. 물론 lemmatization은 stemming만큼 DTM dimension reduction 측면에서 효과적이진 않습니다. 왜냐하면 품사 정보를 여전히 가지고 있어 stemming에 비해 더 적은 단어들이 하나의 단어로 통일되기 때문입니다.

In [9]:
from nltk.stem import WordNetLemmatizer

words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

# lemmatization
n = WordNetLemmatizer()
result = [n.lemmatize(w) for w in words]

# 결과 출력
print(words)
print(result)

['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


In [10]:
# 단어의 품사 정보를 알려주어 다시 출력
print(n.lemmatize('dies', 'v'))
print(n.lemmatize('watched', 'v'))
print(n.lemmatize('has', 'v'))

die
watch
have


### 5.3 Stemming vs. Lemmatization

|비교|Stemming|Lemmatization|
|-|--------|-------------|
|의미|어간 추출|원형 복원|
|접근 방법|정보검색적|언어학적|
|DTM dimension reduction 관점|good|bad|
|의미론적 관점|bad (품사 X)|good (품사 O)|

영어 텍스트의 경우에는 stemming과 lemmatization이 명확하게 구분되어 텍스트 전처리 과정에서 무엇을 사용할지를 결정해야 합니다. 반면 한글 텍스트의 경우에는 형태소 분석 과정에서 stemming과 lemmatization이 함께 이루어진다고 볼 수 있습니다.

## 6. Stopwords Removal (불용어 제거)

### 6.1 개념

**Zipf's Law (지프의 법칙)**

Zipf’s law는 어떠한 자연어 말뭉치 표현에 나타나는 단어들을 그 사용 빈도가 높은 순서대로 나열하였을 때, 모든 단어의 사용 빈도는 해당 단어의 순위에 반비례함을 나타냅니다. 다시 말해, 가장 사용 빈도가 높은 단어는 두번째 단어보다 빈도가 약 두배 높으며, 세번째 단어보다는 빈도가 약 세배 높다는 것입니다. Zipf’s law에 따르면, 문헌집단에 나타나는 단어들의 빈도수를 시각화 하면 long tail distribution이 나타나고, 이때 highly rank된 단어들 즉 고빈도 단어들 중에는 전치사가 대부분입니다.

이러한 단어의 frequency와 단어의 rank의 곱이 constant 즉 일정하다는 Zipf’s law에 기반하여 Luhn은 단어의 출현 빈도 및 순위와 단어의 문헌 식별력 간의 관계를 밝혀냈습니다. 이에 따르면, 최고 한계 빈도와 최저 한계 빈도 안에 속하는 중간 빈도의 단어들이 문헌 내용의 식별력이 크므로 이들을 색인어로 선정하는 것이 바람직합니다. 고빈도 단어들은 기능적인 역할을 하거나 문헌집단 전반에 걸쳐 나타나기 때문에 특정 문헌의 내용을 대표할 수 없고, 저빈도 단어들은 특정 문헌의 내용을 나타낸다고 보기에 너무 적은 빈도 수를 갖기 때문입니다.
 
**Stopwords Removal (불용어 제거)**

Zipf’s law에서 왼쪽에 존재하는 고빈도 단어들을 stopwords라고 합니다. 이러한 stopwords를 모아놓은 것을 stopwords list라고 하고, 이를 활용하여 텍스트 전처리 과정에서 stopwords removal을 하게 됩니다. 영어의 경우 정관사, 전치사 등이 stopwords에 속하고, 한글의 경우 조사 등이 stopwords에 포함됩니다. 불용어 리스트는 정보검색 분야에서 많이 사용된다. 불용어 리스트는 한 라인이 한 단어로 이루어집니다. 영어의 경우 450-500개 정도의 불용어 리스트가 있고, 한국어도 마찬가지로 불용어 리스트를 사용합니다.
 
Stopwords Removal의 목적은 크게 두가지입니다. 단어 정제를 통해 보다 제대로 된 분석을 하기 위함이기도 하고, DTM의 dimension reduction을 통해 분석을 가능하게 하기 위함이기도 합니다. 불용어 리스트를 사용한다는 것은 DTM의 컬럼 수를 줄여주는 것 즉 dimension을 줄여주는 것을 의미합니다. 이를 dimension reduction이라고 합니다. 불용어 리스트를 사용하여 feature들 즉 단어들의 수를 줄여주게 되는데, 그러면 matrix size가 줄어들게 됩니다. 이렇게 불용어 리스트를 사용해서 불용어를 제거하는 것은 dimension reduction에 있어 주요한 역할을 합니다.

### 6.2 NLTK에서 정의한 불용어 리스트 사용하기 (영어)

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [12]:
example = "Family is not an important thing. It's everything."

# 불용어 리스트 생성
stop_words = set(stopwords.words('english')) 

# 단어 토큰화 실시
word_tokens = word_tokenize(example)

# 단어 토큰화 결과로부터 불용어 제거 실시
result = []
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 

# 결과 출력
print(word_tokens) 
print(result) 

['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


NLTK에서 제공하는 불용어 리스트를 활용하여 불용어를 제거한 결과 'is', 'not', 'an'과 같은 단어들이 제거된 것을 볼 수 있습니다.

### 6.3 사용자 정의 불용어 리스트 사용하기

실제로 불용어 제거를 할 때에는 다양한 패키지에서 제공하는 불용어 리스트 뿐만 아니라 해당 코퍼스에 특화되어 있는 사용자 정의 불용어 리스트 또한 사용하는 경우가 많습니다. 일반적으로 불용어 리스트는 txt 파일 혹은 csv 파일 형태로 저장하는데, 여기에는 각각의 불용어가 줄바꿈을 기준으로 한줄에 하나씩 적혀있습니다. 하지만 이번에는 편의상 임의로 불용어 리스트를 생성하여 실습을 진행하도록 하겠습니다.

In [13]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [14]:
example = "고기를 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."

# 불용어 리스트 생성
stop_words = "아무거나 아무렇게나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 하면 아니거든"
stop_words = stop_words.split(' ')

# 단어 토큰화 실시
word_tokens = word_tokenize(example)

# 불용어 제거 실시
result = [word for word in word_tokens if word not in stop_words]

# 결과 출력
print(word_tokens) 
print(result)

['고기를', '아무렇게나', '구우려고', '하면', '안', '돼', '.', '고기라고', '다', '같은', '게', '아니거든', '.', '예컨대', '삼겹살을', '구울', '때는', '중요한', '게', '있지', '.']
['고기를', '구우려고', '안', '돼', '.', '고기라고', '다', '같은', '게', '.', '삼겹살을', '구울', '때는', '중요한', '게', '있지', '.']


**한국어 불용어 리스트 참고 사이트**  
https://www.ranks.nl/stopwords/korean  
https://bab2min.tistory.com/544